In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Write calculation to tag

## Create Client
It is best to log in as a user so we can actually manage the tags we create in the TrendMiner user interface.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"
username = "wdaniels"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Brussels"
)

## Search
As an example, let's calculate the average temperature per batch.

In [3]:
vbs = client.search.value(
    queries=["[CS]BA:ACTIVE.1 = Active"],
    duration="1h",
    calculations={"temp": ("[CS]BA:TEMP.1", "avg")}
)

intervals = vbs.get_results("1M")

## To Series
We can simply use the interval start times as the timestamps for the calculation values.

In [4]:
import pandas as pd

index = [interval.start for interval in intervals]
data = [interval["temp"] for interval in intervals]
ser = pd.Series(index=index, data=data)
ser.head()

2022-12-18 16:18:00+01:00    21.125954
2022-12-18 17:29:00+01:00    21.554201
2022-12-18 18:50:00+01:00    21.360258
2022-12-18 20:11:00+01:00    24.531322
2022-12-18 21:32:00+01:00    19.473837
dtype: float64

If we want the resulting tag to return to a default value (e.g., 0), in between batches, we can use the interval end timestamps too.

In [5]:
index = index + [interval.end for interval in intervals]
data = data + [0 for interval in intervals]
ser = pd.Series(index=index, data=data)
ser.sort_index(inplace=True)
ser.head()

2022-12-18 16:18:00+01:00    21.125954
2022-12-18 17:18:00+01:00     0.000000
2022-12-18 17:29:00+01:00    21.554201
2022-12-18 18:39:00+01:00     0.000000
2022-12-18 18:50:00+01:00    21.360258
dtype: float64

## To Tag
We can now create our stepped tag and give it its data

In [6]:
tag = client.tag(
    name="BA:TEMP-AVG.1",
    description="average batch temperature",
    tag_type="DISCRETE",
    data=ser
)

In [7]:
# HIDDEN - set unique tag name
tag.name = "BA:" + str(int(client.time.now().timestamp()))

And finally upload it to the appliance as a csv-imported tag

In [8]:
client.io.tag.post(tags=[tag])

We can overwrite a tag by changing the data and posting again:

In [9]:
tag.data = tag.data*10
client.io.tag.post([tag])

We can delete a tag, but remember we can then **never use this tag name again**

In [10]:
client.io.tag.delete(tag)